In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import warnings
from tqdm import tqdm

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, cross_val_predict
import sklearn
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.metrics import f1_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import xgboost

warnings.filterwarnings('ignore')
plt.style.use('seaborn')

## Loading data

In [33]:
#Convert Personality to DataFrame
personality = loadmat('data/Dt_Personality.mat')
personality = personality['Personality']
big5 = pd.DataFrame(personality, columns=['extro', 'agree', 'cons', 'stability', 'open'])
#big5.head()

In [34]:
#Load MATLAB file for ECG"
ECG = loadmat('data/Dt_ECGFeatures.mat')
ECG_feats_mat = ECG['ECGFeatures_58']

In [35]:
def unfold_mat(mat, feats=None):
    #Returns list with each index being participant.
    #Each item has dataframe with shape video x feature (row is video, columns are features)
    lst = []
    for i in mat:
        d = pd.DataFrame(i, columns=feats)
        d.index += 1
        lst.append(d)
    
    return lst

def statistical_measurements(suffix):
    #Some modalities have statistical measurements, they are always the same 6 items.
    cols = []
    names = ['mean', 'std', 'skewness', 'kurtosis', '%time_over_mean+std', '%time_over_mean-std']
    for i in range(6):
        cols.append(names[i]+'_'+suffix)
    return cols

In [36]:
#Isolate ECG (Heart Monitor) Features
ECG_cols = ['low_freq_PSD' for x in range(10)]
[ECG_cols.append('slow_response_PSD') for x in range(4)]
[ECG_cols.append(x) for x in statistical_measurements('IBI')]
[ECG_cols.append(x) for x in statistical_measurements('HR')]
[ECG_cols.append(x) for x in statistical_measurements('HRV')]

ECG_feats = unfold_mat(ECG_feats_mat[0], feats=ECG_cols)

#Remove columns, where all rows are 0.0 or NaN - ECG had a lot of useless readings.
for i in range(len(ECG_feats)):
    ECG_feats[i] = ECG_feats[i].drop('low_freq_PSD', axis=1)
    ECG_feats[i] = ECG_feats[i].drop('slow_response_PSD', axis=1)
    ECG_feats[i] = ECG_feats[i].drop('mean_IBI', axis=1)
    ECG_feats[i] = ECG_feats[i].drop('std_IBI', axis=1)
    ECG_feats[i] = ECG_feats[i].drop('skewness_IBI', axis=1)
    ECG_feats[i] = ECG_feats[i].drop('kurtosis_IBI', axis=1)
    

#ECG_feats[0].head()

In [37]:
#Load EEG features
EEG = loadmat('data/Dt_EEGFeatures.mat')
EEG_feats_mat = EEG['EEGFeatures_58']

#Isolate EEG features
EEG_cols = []
[EEG_cols.append(x) for x in ['avg_first_deriv', 'prop_neg_diff_samples', 'mean_num_peaks', 'mean_deriv_inv_signal', 'avg_num_peaks_inv_signal']]
[EEG_cols.append(x) for x in statistical_measurements('EEG')]
[EEG_cols.append('attention') for x in range(11)]
[EEG_cols.append('meditation') for x in range(11)]
[EEG_cols.append('alpha') for x in range(11)]
[EEG_cols.append('beta') for x in range(11)]
[EEG_cols.append('delta') for x in range(11)]
[EEG_cols.append('gamma') for x in range(11)]
[EEG_cols.append('theta') for x in range(11)]

EEG_feats = unfold_mat(EEG_feats_mat[0], feats=EEG_cols)
#EEG_feats[0].head()

In [38]:
#Load GSR features
GSR = loadmat('data/Dt_GSRFeatures.mat')
GSR_feats_mat = GSR['GSRFeatures_58']
GSR_fails_mat = GSR['GSRFailures_58']

GSR_cols = ['mean_resistance',
            'mean_derivative',
            'mean_differential_neg_values',
            'proportion_neg_derivative_samples',
            'num_local_signal_minima',
            'avg_rising_time',
            'spectral_power_Hz',
            'zero_crossing_slow',
            'zero_crossing_very_slow',
            'mean_scsr',
            'scvsr_peak_mag']


GSR_feats = unfold_mat(GSR_feats_mat[0])
#GSR_feats[0].head()

In [39]:
#Load facial landmark features
EMO = loadmat('data/Dt_EMOFeatures.mat')
EMO_feats_mat = EMO['EMOFeatures_58']

EMO_cols = []
[EMO_cols.append(x) for x in statistical_measurements('verti_upper_lip')]
[EMO_cols.append(x) for x in statistical_measurements('verti_lower_lip')]
[EMO_cols.append(x) for x in statistical_measurements('horis_left_lip')]
[EMO_cols.append(x) for x in statistical_measurements('verti_left_lip')]
[EMO_cols.append(x) for x in statistical_measurements('horis_right_lip')]
[EMO_cols.append(x) for x in statistical_measurements('verti_right_lip')]
[EMO_cols.append(x) for x in statistical_measurements('right_eyebrow')]
[EMO_cols.append(x) for x in statistical_measurements('left_eyebrow')]
[EMO_cols.append(x) for x in statistical_measurements('right_cheek')]
[EMO_cols.append(x) for x in statistical_measurements('left_cheek')]
[EMO_cols.append(x) for x in statistical_measurements('right_lid')]
[EMO_cols.append(x) for x in statistical_measurements('left_lid')]

EMO_feats = unfold_mat(EMO_feats_mat[0], feats=EMO_cols)
#EMO_feats[0].head()

## Data preprocessing

In [40]:
def get_group(modality, group):
    #Returns a subset of modality after grouped movie clip
    #ex: get_group(EEG_feats, LAHV) -> list with dataframes, each is a participant, each row movie
    out = []
    for i in range(len(modality)):
        out.append(modality[i].loc[group])
    return out

def trait_binary(trait):
    #Converts personality trait to 0/1 (low/high) depending on median.
    #Returns a 1D numpy array
    median = np.median(big5.loc[:][trait].values)
    mask = big5.loc[:][trait]>median
    return mask.astype(int).values
    

def format_train_data(modality, trait, shuffle=False):
    #Creates input vectors and output vectors for modality and trait combination.
    vector_x = []
    vector_y = []
    
    bool5 = trait_binary(trait)
    
    for i, participant in enumerate(modality):
        participant.replace([np.inf, -np.inf], np.nan, inplace=True)
        participant = participant.dropna()
        y = bool5[i]
        
        for clip in participant.index:
            vector_x.append(participant.loc[clip].values)
            vector_y.append(y)
    
    vector_y = np.array(vector_y)
    vector_x = np.array(vector_x)
    if shuffle:
        vector_x, vector_y = sklearn.utils.shuffle(vector_x, vector_y, random_state=42)
        
    return vector_x, vector_y

def preprocess(features, target_trait):
    #Function, which returns standardized PCA on X and y
    pca = PCA(n_components=0.99)
    X, y = format_train_data(features, target_trait, shuffle=True)
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    x_pca = pca.fit_transform(X)
    
    return x_pca, y

def results(classifier):
    #Creates dataframe with unimodal results.
    mats = { #Dict with column names corresponding to features.
        'EEG': EEG_feats,
        'ECG': ECG_feats,
        'EMO': EMO_feats,
        'GSR': GSR_feats
    }
    scores = []
    all_targets = []
    all_preds = []
    #Initialise dataframe
    df = pd.DataFrame(columns=big5.keys(), index=['ECG', 'EEG', 'GSR', 'EMO'])
    for trait in tqdm(list(df.keys())):
        for modal in list(df.index):
            #For each combination of modality and trait:
            X, y = preprocess(mats[modal], trait) #
            score = round(np.mean(cross_val_score(classifier, X, y, scoring='f1', cv=10)),2)
            df[trait][modal] = score
            scores.append(score)
            all_targets.append(y)
            all_preds.append(cross_val_predict(classifier, X, y, cv=10, method='predict_proba'))

    print("Mean score:", np.mean(scores))
    print("STD score:", np.std(scores))
    
    return df, (np.array(all_targets), np.array(all_preds))

## Models

### Logistic Regression

In [41]:
warnings.filterwarnings('ignore') #uncomment to turn on warnings
logreg = LogisticRegressionCV(random_state=42, max_iter=2500, scoring='f1', n_jobs=4)
df, log_targs = results(logreg)
df

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.52s/it]

Mean score: 0.544
STD score: 0.09911609354691095


,extro,agree,cons,stability,open
ECG,0.63,0.52,0.56,0.62,0.48
EEG,0.63,0.42,0.48,0.64,0.43
GSR,0.57,0.49,0.49,0.59,0.25
EMO,0.66,0.66,0.62,0.56,0.58


### Random Forest Classifier

In [42]:
rfc = RandomForestClassifier(n_estimators=500, n_jobs=4, random_state=42)
df, rf_targs = results(rfc)
df

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:23<00:00, 88.67s/it]

Mean score: 0.612
STD score: 0.11404385121522334


,extro,agree,cons,stability,open
ECG,0.72,0.72,0.72,0.72,0.65
EEG,0.62,0.44,0.41,0.64,0.37
GSR,0.59,0.54,0.58,0.61,0.43
EMO,0.76,0.71,0.69,0.67,0.65


### XGBoost Test

In [43]:
xgb = xgboost.XGBClassifier(n_estimators=500, n_jobs=4, random_state=42)
df, xgb_targs = results(xgb)
df

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

[18:49:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:49:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:49:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:49:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[18:49:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:49:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:49:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:49:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[18:50:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:50:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:50:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:50:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

 20%|████████████████▊                                                                   | 1/5 [01:34<06:19, 94.95s/it]

...

[18:56:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:56:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:56:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:56:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[18:56:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:56:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:56:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:56:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [08:13<00:00, 98.74s/it]

Mean score: 0.624
STD score: 0.092


,extro,agree,cons,stability,open
ECG,0.71,0.69,0.69,0.71,0.63
EEG,0.61,0.49,0.49,0.63,0.48
GSR,0.57,0.54,0.57,0.59,0.47
EMO,0.75,0.75,0.69,0.72,0.7


### Multi-Layer Perceptron 

In [44]:
# solver='lbfgs' is supposed to be used for small datasets, but i get better results using the default 'adam'

mlp = MLPClassifier(random_state=42, max_iter=2000)
df, mlp_targs = results(mlp)
df

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [49:02<00:00, 588.42s/it]

Mean score: 0.6475000000000002
STD score: 0.11058367872339932


,extro,agree,cons,stability,open
ECG,0.73,0.71,0.72,0.72,0.65
EEG,0.56,0.52,0.52,0.62,0.47
GSR,0.59,0.56,0.58,0.61,0.46
EMO,0.82,0.81,0.76,0.77,0.77


In [45]:
#Decision fusion

#ECG
a_ecg = model_weights_ecg #stand-in for actual weights
p_ecg = model_post_prob_ecg #stand-in for posterior prob

#EEG
a_eeg = model_weights _eeg
p_eeg = model.model_post_prob_eeg

#GSR
a_gsr = model_weights_gsr 
p_gsr = model_post_prob_gsr

#EMO
a_emo = model_weights_emo 
p_emo = model_post_prob_emo


t_ecg = (a_ecg * f1_ecg) / (a_ecg * f1_ecg) + (a_eeg * f1_eeg) + (a_gsr * f1_gsr) + (a_emo * f1_emo)
t_eeg = (a_eeg * f1_eeg) / (a_ecg * f1_ecg) + (a_eeg * f1_eeg) + (a_gsr * f1_gsr) + (a_emo * f1_emo)
t_gsr = (a_gsr * f1_gsr) / (a_ecg * f1_ecg) + (a_eeg * f1_eeg) + (a_gsr * f1_gsr) + (a_emo * f1_emo)
t_emo = (a_emo * f1_emo) / (a_ecg * f1_ecg) + (a_eeg * f1_eeg) + (a_gsr * f1_gsr) + (a_emo * f1_emo)

df_ecg = a_ecg * (t_ecg * p_ecg)
df_eeg = a_eeg * (t_eeg * p_eeg)
df_ecg = a_gsr * (t_gsr * p_gsr)
df_ecg = a_emo * (t_ecg * p_emo)

result_df = df_ecg + df_eeg + df_gsr + df_emo

SyntaxError: invalid syntax (<ipython-input-45-8ba32d4fb9f8>, line 8)

## Plotting results

In [ ]:
def plot_roc_cur(fper, tper, label="", title="plot title"):
    plt.plot(fper, tper, label=label)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc='lower right')
    
def create_roc_curve(preds, targets, label="", title=""):
    all_fpr, all_tpr = [], []
    fpr, tpr, thresholds = roc_curve(preds[0], targets[0][:,1])
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_fpr = np.array(all_fpr)
    all_tpr = np.array(tpr)
    
    plot_roc_cur(all_fpr[0], all_tpr, label=label, title=title)
    
create_roc_curve(log_targs[0], log_targs[1], label="Logistic Regression")
create_roc_curve(rf_targs[0], rf_targs[1], label="Random Forest")
create_roc_curve(xgb_targs[0], xgb_targs[1], label="XGBoost")
create_roc_curve(mlp_targs[0], mlp_targs[1], label="Multilayer Perceptron", title="ROC Curves for implemented models")